In [1]:
import numpy as np

In [2]:
answers = [
    [3, 0],
    [5, 1],
]
maxx_const = int(np.random.rand() * 20)

In [3]:
def alex(opponent_last, memory=None):
    return 1, memory;

def bob(opponent_last, memory=None):
    return 0, memory;

def clara(opponent_last, memory=None):
    if opponent_last is None:
        return 1, memory
    return opponent_last, memory

def denis(opponent_last, memory=None):
    if opponent_last is None:
        return 1, memory
    return 1 if opponent_last == 0 else 0, memory

def emma(opponent_last, memory=None):
    if memory is None:
        memory = {'round': 0}
    
    memory['round'] += 1
    
    if memory['round'] % 20 == 0:
        return 0, memory
    else:
        return 1, memory

def frida(opponent_last, memory=None):
    if memory is None:
        memory = {'opponent_always_1': True}
    
    if opponent_last is None:
        return 1, memory
    
    if opponent_last == 0:
        memory['opponent_always_1'] = False
    
    if memory['opponent_always_1']:
        return 1, memory
    else:
        return 0, memory

def george(opponent_last, memory=None):
    if memory is None:
        memory = { 'count': 0 }
        return 0, memory
    
    if opponent_last == 1:
        memory['count'] += 1
    
    if memory['count'] >= 10 and opponent_last == 1:
        memory['count'] = 0
        return 0, memory
    else:
        return opponent_last, memory

In [41]:
def hank(opponent_last, memory=None):
    return (np.random.rand() < 0.5) * 1, memory

def ivan(opponent_last, memory=None):
    return (np.random.rand() < 0.9) * 1, memory

def jack(opponent_last, memory=None):
    if opponent_last is None or opponent_last == 0:
        return 0, memory
    return (np.random.rand() < 0.75) * 1, memory

def kevin(opponent_last, memory=None):
    if opponent_last is None:
        return 0, memory
    if np.random.rand() < 0.75:
        return opponent_last, memory
    else:
        return 1 - opponent_last, memory

def lucas(opponent_last, memory=None):
    if memory is None:
        lucas_c = int(np.random.rand() * 10) + 1
        memory = { 'count': 1, 'lucas_c' : lucas_c }
        return 0, memory

    memory['count'] += 1
    
    if memory['count'] >= memory['lucas_c']:
        memory['count'] = 0
        return 1, memory
    else:
        return 0, memory
        
def maxx(opponent_last, memory=None):
    global maxx_const
    if memory is None:
        memory = { 'count': 0, 'hod': 0}
        return memory['hod'], memory

    memory['count'] += 1
    
    if memory['count'] >= maxx_const:
        memory['count'] = 0
        max_const = int(np.random.rand() * 20)
        memory['hod'] = 1 - memory['hod']
        
    return memory['hod'], memory

    
def natan(opponent_last, memory=None):
    if memory is None:
        memory = {'goodwill': 10, 'anger': 0}
        return 0, memory

    if opponent_last == 1:
        memory['goodwill'] = max(0, memory['goodwill'] - 1)
        memory['anger'] = min(10, memory['anger'] + 1)
        
        if memory['goodwill'] <= 0:
            prob = memory['anger'] / 10
            if np.random.rand() < prob:
                return 1, memory
    
    elif np.random.rand() < 0.1:
        memory['goodwill'] = memory['goodwill'] + 1
        memory['anger'] = memory['anger'] - 1
    
    return 0, memory

In [42]:
def mode(scores):
    if len(scores) <= 1:
        return round(np.mean(scores)) if scores else 0
    
    k = int(np.log2(1 + len(scores)))
    
    min_val = min(scores)
    max_val = max(scores)
    
    if max_val - min_val < 1:
        return round(np.mean(scores))
    
    intervals = []
    interval_width = (max_val - min_val) / k
    
    for i in range(k):
        start = min_val + i * interval_width
        end = min_val + (i + 1) * interval_width
        intervals.append({'start': start, 'end': end, 'count': 0, 'sum': 0})
    
    for score in scores:
        for i, interval in enumerate(intervals):
            if interval['start'] <= score < interval['end'] or (i == k-1 and score == interval['end']):
                interval['count'] += 1
                interval['sum'] += score
                break
    
    max_interval = max(intervals, key=lambda x: x['count'])
    
    if max_interval['count'] > 0:
        return round(max_interval['sum'] / max_interval['count'])
    else:
        return round(np.mean(scores))

In [43]:
def play_game(strategy_a, strategy_b):
    score_a = 0
    score_b = 0
    last_a = None
    last_b = None
    memory_a = None
    memory_b = None

    max_dominant_a = 0
    max_dominant_b = 0
    current_dominant_a = 0
    current_dominant_b = 0
    
    for i in range(200):
        choice_a, memory_a = strategy_a(last_b, memory_a)
        choice_b, memory_b = strategy_b(last_a, memory_b)
        
        points_a = answers[choice_a][choice_b]
        points_b = answers[choice_b][choice_a]
        
        score_a += points_a
        score_b += points_b
        
        last_a = choice_a
        last_b = choice_b

        if points_a == 5 and points_b == 0:
            current_dominant_a += 1
            current_dominant_b = 0
            max_dominant_a = max(max_dominant_a, current_dominant_a)
        elif points_b == 5 and points_a == 0:
            current_dominant_b += 1
            current_dominant_a = 0
            max_dominant_b = max(max_dominant_b, current_dominant_b)
        else:
            current_dominant_a = 0
            current_dominant_b = 0
    
    return score_a, score_b, max_dominant_a, max_dominant_b

In [44]:
def play_game_st(strategy_a, strategy_b):
    scores_a = []
    scores_b = []
    dominants_a = []
    dominants_b = []
    
    for i in range(100):
        score_a, score_b, dom_a, dom_b = play_game(strategy_a, strategy_b)
        scores_a.append(score_a)
        scores_b.append(score_b)
        dominants_a.append(dom_a)
        dominants_b.append(dom_b)
        
    return scores_a, scores_b, np.mean(dominants_a), np.mean(dominants_b)

In [45]:
strategies = [
    ("alex", alex),
    ("bob", bob),
    ("clara", clara),
    ("denis", denis),
    ("emma", emma),
    ("frida", frida),
    ("george", george),
    ("hank", hank),
    ("ivan", ivan),
    ("jack", jack),
    ("kevin", kevin),
    ("lucas", lucas),
    ("maxx", maxx),
    ("natan", natan)
]

strategies_1 = [
    ("alex", alex),
    ("bob", bob),
    ("clara", clara),
    ("denis", denis),
    ("emma", emma),
    ("frida", frida),
    ("george", george),
]

strategies_2 = [
    ("hank", hank),
    ("ivan", ivan),
    ("jack", jack),
    ("kevin", kevin),
    ("lucas", lucas),
    ("maxx", maxx),
    ("natan", natan)
]


In [46]:
total_scores = {name: 0 for name, _ in strategies}
dominant_series = {name: 0 for name, _ in strategies}
pairwise_scores = {}
pairwise_dominant = {}
pairwise_median = {}
pairwise_mode = {}
pairwise_variance ={}

for i, (name1, strat1) in enumerate(strategies):
    for j, (name2, strat2) in enumerate(strategies):
        if name1 in [name for name, _ in strategies_2] or name2 in [name for name, _ in strategies_2]:
            scores1, scores2, avg_dom1, avg_dom2 = play_game_st(strat1, strat2)
            mean_a = round(np.mean(scores1))
            mean_b = round(np.mean(scores2))
            pairwise_scores[(name1, name2)] = (mean_a, mean_b)
            pairwise_dominant[(name1, name2)] = (round(avg_dom1), round(avg_dom2))

            scores1_sorted = sorted(scores1)
            scores2_sorted = sorted(scores2)
            pairwise_median[(name1, name2)] = (round(np.median(scores1_sorted)), round(np.median(scores2_sorted)))

            pairwise_mode[(name1, name2)] = (mode(scores1), mode(scores2))

            variance_a = np.var(scores1, ddof=1)
            variance_b = np.var(scores2, ddof=1)
            pairwise_variance[(name1, name2)] = (variance_a, variance_b)
        else:
            score1, score2, dom1, dom2 = play_game(strat1, strat2)
            pairwise_scores[(name1, name2)] = (score1, score2)
            pairwise_dominant[(name1, name2)] = (dom1, dom2)
            
        score1, score2 = pairwise_scores[(name1, name2)]
        dom1, dom2 = pairwise_dominant[(name1, name2)]
            
        total_scores[name1] += score1
        total_scores[name2] += score2
        dominant_series[name1] += dom1
        dominant_series[name2] += dom2

In [47]:
print("\n" + "=" * 70)
print("ТАБЛИЦА 1: ОБЩЕЕ ЧИСЛО ОЧКОВ")
print("=" * 70)

print(f"{'':<12}", end="")
for name, _ in strategies:
    print(f"{name:<8}", end="")
print("Total")

for name1, _ in strategies:
    print(f"{name1:<12}", end="")
    for name2, _ in strategies:
        score1, score2 = pairwise_scores[(name1, name2)]
        print(f"{score1:<8}", end="")
    print(f"{total_scores[name1]:<8}")

print("\n" + "=" * 70)
print("ТАБЛИЦА 2: ДЛИНА НАИБОЛЬШЕЙ ДОМИНИРУЮЩЕЙ СЕРИИ")
print("=" * 70)

print(f"{'':<12}", end="")
for name, _ in strategies:
    print(f"{name:<8}", end="")
print("Total")

for name1, _ in strategies:
    print(f"{name1:<12}", end="")
    for name2, _ in strategies:
        dom1, dom2 = pairwise_dominant[(name1, name2)]
        print(f"{dom1:<8}", end="")
    print(f"{dominant_series[name1]:<8}")

print("\n" + "=" * 70)
print("РЕЙТИНГ СТРАТЕГИЙ ПО ОБЩЕМУ ЧИСЛУ ОЧКОВ")
print("=" * 70)
sorted_by_score = sorted(total_scores.items(), key=lambda x: x[1], reverse=True)
for rank, (name, score) in enumerate(sorted_by_score, 1):
    print(f"{rank:2}. {name:<10}: {score:>6} очков")

print("\n" + "=" * 70)
print("РЕЙТИНГ СТРАТЕГИЙ ПО ДОМИНИРУЮЩИМ СЕРИЯМ")
print("=" * 70)
sorted_by_dominant = sorted(dominant_series.items(), key=lambda x: x[1], reverse=True)
for rank, (name, dom) in enumerate(sorted_by_dominant, 1):
    print(f"{rank:2}. {name:<10}: {dom:>6} серий")

print("\n" + "=" * 70)
print("ТАБЛИЦА 3: МЕДИАНЫ")
print("=" * 70)

print(f"{'':<12}", end="")
for name, _ in strategies:
    print(f"{name:<8}", end="")
print()

for name1, _ in strategies:
    print(f"{name1:<12}", end="")
    for name2, _ in strategies:
        if (name1, name2) in pairwise_median:
            score1, score2 = pairwise_median[(name1, name2)]
            print(f"{score1:<8}", end="")
        elif (name2, name1) in pairwise_median:
            score2, score1 = pairwise_median[(name2, name1)]
            print(f"{score1:<8}", end="")
        else:
            print(f"{'—':<8}", end="")
    print() 
    

print("\n" + "=" * 70)    
print("ТАБЛИЦА 4: МОДА")
print("=" * 70)

print(f"{'':<12}", end="")
for name, _ in strategies:
    print(f"{name:<8}", end="")
print()

for name1, _ in strategies:
    print(f"{name1:<12}", end="")
    for name2, _ in strategies:
        if (name1, name2) in pairwise_mode:
            score1, score2 = pairwise_mode[(name1, name2)]
            print(f"{score1:<8}", end="")
        elif (name2, name1) in pairwise_mode:
            score2, score1 = pairwise_mode[(name2, name1)]
            print(f"{score1:<8}", end="")
        else:
            print(f"{'—':<8}", end="")
    print() 
    
print("\n" + "=" * 70)    
print("ТАБЛИЦА 4: ДИСПЕРСИЯ")
print("=" * 70)

print(f"{'':<12}", end="")
for name, _ in strategies:
    print(f"{name:<8}", end="")
print()

for name1, _ in strategies:
    print(f"{name1:<12}", end="")
    for name2, _ in strategies:
        if (name1, name2) in pairwise_variance:
            score1, score2 = pairwise_variance[(name1, name2)]
            print(f"{score1:<8.2f}", end="")
        elif (name2, name1) in pairwise_variance:
            score2, score1 = pairwise_variance[(name2, name1)]
            print(f"{score1:<8.2f}", end="")
        else:
            print(f"{'—':<8}", end="")
    print() 


ТАБЛИЦА 1: ОБЩЕЕ ЧИСЛО ОЧКОВ
            alex    bob     clara   denis   emma    frida   george  hank    ivan    jack    kevin   lucas   maxx    natan   Total
alex        200     1000    200     996     240     200     280     601     279     402     405     760     616     240     12823   
bob         0       600     597     0       30      597     600     300     60      600     453     431     312     600     10334   
clara       200     602     200     450     231     200     593     449     256     599     454     530     422     602     11552   
denis       1       1000    450     400     40      989     474     449     83      524     451     631     508     476     12963   
emma        190     980     226     940     220     901     305     584     271     413     410     765     600     266     14174   
frida       200     602     200     9       51      200     599     303     71      600     450     405     314     602     9216    
george      180     600     588     419   